In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
import tensorflow as tf
from tqdm import tqdm

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D,MaxPooling2D,UpSampling2D,Input,BatchNormalization,LeakyReLU

from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

from IPython import display


from keras.preprocessing.image import img_to_array, load_img
from skimage.color import lab2rgb, rgb2lab


In [ ]:
def extract_single_dim_from_LAB_convert_to_RGB(image,idim):
        '''
        image is a single lab image of shape (None,None,3)
        '''
        z = np.zeros(image.shape)
        if idim != 0 :
            z[:,:,0]=80 ## I need brightness to plot the image along 1st or 2nd axis
        z[:,:,idim] = image[:,:,idim]
        z = lab2rgb(z)
        return(z)

In [ ]:
ImagePath="./TESTSimpleColor/TemporalDataSet/dataset_updated/training_set/painting/"


image = img_to_array(load_img(ImagePath+"0001.jpg",target_size=(224,224))) / 255
lab_image = rgb2lab(image)
lab_image_norm = (lab_image + [0, 128, 128]) / [100, 255, 255]

print(lab_image.shape)

lab_l = extract_single_dim_from_LAB_convert_to_RGB(lab_image,0)
lab_a = extract_single_dim_from_LAB_convert_to_RGB(lab_image,1)
lab_db = extract_single_dim_from_LAB_convert_to_RGB(lab_image,2)
    
        # Plot the results
fig, axes = plt.subplots(ncols=3, figsize=(12, 4))
data = [('L: lightness', lab_l), ('a: green-magenta channel', lab_a), ('b: blue-yellow channel', lab_db)]
    
for ax, (title, img) in zip(axes, data):
            ax.set_title(title)
            ax.imshow(img)
            ax.axis('off')
    
fig.tight_layout()
plt.show()

In [ ]:
print(os.listdir("./TESTSimpleColor/TemporalDataSet/dataset_updated/"))



HEIGHT=224
WIDTH=224
ImagePath="./TESTSimpleColor/TemporalDataSet/dataset_updated/training_set/painting/"

def ExtractInput(path):
    X_img=[]
    y_img=[]
    for imageDir in os.listdir(ImagePath):
        try:
            image = img_to_array(load_img(ImagePath+"0001.jpg",target_size=(224,224))) / 255
            lab_image = rgb2lab(image)
            lab_image_norm = (lab_image + [0, 128, 128]) / [100, 255, 255]
            X_img.append(lab_image_norm[:,:,0])
            y_img.append(lab_image_norm[:,:,1:])
        except:
            pass
    X_img = np.array(X_img)
    y_img = np.array(y_img)
    
    return X_img,y_img

x_train,y_train = ExtractInput(ImagePath)

def GenerateInputs(X_,y_):
    X_ = list(map(lambda x: x.reshape(1,224,224,1),X_))
    y_ = list(map(lambda x: x.reshape(1,224,224,2),y_))
    return X_,y_

x_train,y_train = GenerateInputs(x_train,y_train)
x_train = np.array(x_train)
y_train = np.array(y_train)


In [ ]:
def adam_optimizer():
    return Adam(learning_rate=0.0002, beta_1=0.5)

In [ ]:
def make_generator_model():
    model = Sequential([
    Input(shape=(HEIGHT, WIDTH,1)),
    Conv2D(16,(3,3),padding='same',strides=1),
    LeakyReLU(),
    Conv2D(32,(3,3),padding='same',strides=1),
    Conv2D(32,(3,3),padding='same',strides=1),
    
    LeakyReLU(),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2),padding='same'),
    
    Conv2D(64,(3,3),padding='same',strides=1),
    LeakyReLU(),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2),padding='same'),
    
    Conv2D(128,(3,3),padding='same',strides=1),
    LeakyReLU(),
    BatchNormalization(),
    
    Conv2D(256,(3,3),padding='same',strides=1),
    LeakyReLU(),
    BatchNormalization(),
    
    UpSampling2D((2, 2)),
    Conv2D(128,(3,3),padding='same',strides=1),
    LeakyReLU(),
    BatchNormalization(),
    
    UpSampling2D((2, 2)),
    Conv2D(64,(3,3), padding='same',strides=1),
    LeakyReLU(),
    
    Conv2D(64,(3,3), padding='same',strides=1),
    LeakyReLU(),
    BatchNormalization(),
    
    Conv2D(32,(3,3),padding='same',strides=1),
    LeakyReLU(),
    #BatchNormalization(),
    
    Conv2D(2,(3,3), activation='tanh',padding='same',strides=1),
    ])

    model.compile(loss='binary_crossentropy', optimizer=adam_optimizer())

    return model

In [ ]:

generator = make_generator_model()

noise = x_train[1]
generated_image = generator(noise, training=False)

img = np.zeros((WIDTH,HEIGHT,3)).reshape(224,224,3)
print(img.shape)
img[:,:,0] =x_train[0].reshape(224,224)
img[:,:,1:] = generated_image.numpy().reshape(224,224,2)
img = (img * [100,255,255]) - [0,128,128]
rgb_img = lab2rgb(img)
plt.imshow(rgb_img)
plt.show()

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential([
        Input(shape=(HEIGHT, WIDTH,2)),
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1)
    ])
    model.compile(loss='binary_crossentropy', optimizer=adam_optimizer())

    return model

In [ ]:
generator.summary()

In [ ]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print (decision)

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
discriminator.summary()

In [ ]:
def create_gan(discriminator, generator):
    discriminator.trainable=False
    gan_input = Input(shape=(224,224,1))
    x = generator(gan_input)
    gan_output= discriminator(x)
    gan= Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan
gan = create_gan(discriminator,generator)


In [ ]:
gan.summary()

In [ ]:

def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4, 4))

  for i,(lmap,img) in enumerate(list(zip(test_input,predictions))):
      plt.subplot(4, 4, i+1)
      img = np.zeros((WIDTH,HEIGHT,3)).reshape(224,224,3)
      
      img[:,:,0] =lmap.reshape(224,224)
      img[:,:,1:] = img.reshape(224,224,2)
      img = (img * [100,255,255]) - [0,128,128]
      rgb_img = lab2rgb(img)

    


      plt.imshow(rgb_img)
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

seed = x_train[:15]

In [ ]:
def training(epochs=1, batch_size=128):
    
    #Loading the data
    
    
    # Creating GAN
    generator= make_generator_model()
    discriminator= make_discriminator_model()
    gan = create_gan(discriminator, generator)
    
    for e in range(1,epochs+1 ):
        print("Epoch %d" %e)
        for _ in tqdm(range(batch_size)):
        #generate  random noise as an input  to  initialize the  generator
            A = len(x_train)
            image_batch =x_train[np.random.randint(low=0,high=A,size=batch_size)]
                
            # Generate fake MNIST images from noised input
            generated_images = generator.predict(image_batch)
            
            # Get a random set of  real images
            
            
            #Construct different batches of  real and fake data 
            X= np.concatenate([image_batch, generated_images])
            
            # Labels for generated and real data
            y_dis=np.zeros(2*batch_size)
            y_dis[:batch_size]=0.9
            
            #Pre train discriminator on  fake and real data  before starting the gan. 
            discriminator.trainable=True
            discriminator.train_on_batch(X, y_dis)
            
            #Tricking the noised input of the Generator as real data
            image_batch2 =x_train[np.random.randint(low=0,high=A,size=batch_size)]
            y_gen = np.ones(batch_size)
            
            # During the training of gan, 
            # the weights of discriminator should be fixed. 
            #We can enforce that by setting the trainable flag
            discriminator.trainable=False
            
            #training  the GAN by alternating the training of the Discriminator 
            #and training the chained GAN model with Discriminator’s weights freezed.
            gan.train_on_batch(image_batch2, y_gen)
            
        if e == 1 or e % 2 == 0:
           
            generate_and_save_images( generator,e,seed)
training(10,128)